In [109]:
import sys
import os
sys.path.insert(0, os.getcwd())


from src.docx2graph.utils import build_knowledge_graph, draw_graph, get_triples_from_dcx, node_link_data
from langchain_core.documents.base import Document
from docx_parser.document_parser import DOCXParser

doc_path="/Applications/programming/kg_llm/dozor/install.docx"
parser = DOCXParser()
parser.parse(doc_path)


triples, _ = get_triples_from_dcx(parser.get_lines_with_meta())
G = build_knowledge_graph(triples)
link_data_json = node_link_data(G)


In [110]:
from langchain_community.graphs.graph_document import GraphDocument, Node,Relationship

In [111]:
nodes = []
relationships = []
source = []
for triplet in triples:
    src = Node(id=triplet[0].text[:100], 
               type=triplet[0].__class__.__name__, 
               properties = {'text': triplet[0].text})
    
    target = Node(id=triplet[2].text[:100], 
                  type=triplet[2].__class__.__name__, 
                  properties = {'text': triplet[2].text})
    nodes.append(src)
    nodes.append(target)
    
    relationships.append(
        Relationship(source=src, 
                     target=target, 
                     type=triplet[1])
    )
    
    source = Document(page_content="content")
    graph_documents = GraphDocument(nodes=nodes, relationships=relationships, source=source)

In [112]:
os.environ["NEO4J_URI"] ="bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "kodin"
os.environ["NEO4J_PASSWORD"] = "12345678"

In [113]:
from langchain_community.graphs import Neo4jGraph
graph = Neo4jGraph(database='dozortedocx')

In [114]:
graph.add_graph_documents(
    [graph_documents],
    baseEntityLabel=True,
    include_source=True
)